In [33]:
import bagpy
from bagpy import bagreader
import os
import pandas as pd
from dateutil.parser import parse as parsedate
import matplotlib.pyplot as plt
from rosbags.highlevel import AnyReader as RosBagReader
from pathlib import Path
import logging
from dotenv import dotenv_values
from dotenv import load_dotenv
import ast 
from functions import *
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency, chi2

%load_ext autoreload
%autoreload 2

config = dotenv_values()
data_path = os.path.normpath(config['DATA_PATH'])
os.environ['DATA_PATH'] = data_path
print(f"DATA_PATH is set to: {os.environ['DATA_PATH']}")

rat_number = 7
date = '240826'

messages, bag_file = determine_trial_info(rat_number, date)

extracted_biases_file_path = os.path.join(os.path.dirname(bag_file), 'extracted_biases.csv')

#check whether the files aleady exist. If they exist read them if not create them.
if os.path.exists(extracted_biases_file_path):
    #read the extracted bias csv file that contains trial information
    extracted_biases = pd.read_csv(extracted_biases_file_path)
    #read the trial type summary csv file
    csv_trial_type_summary = os.path.join(os.path.dirname(bag_file), 'trial_type_summary.csv')
    trial_type_summary = pd.read_csv(csv_trial_type_summary)
    print('The dataframes have been created previously')
else:
    filtered_masseages = filter_start_of_trial_messages(messages)

    final_messages = remove_specific_message(filtered_masseages)

    df = create_dataframe_from_messages(final_messages)

    #seperate the 'Left Cue', 'Right Cue', 'Sound Cue' of each Trial (row) and pu them in three new columns
    df[['Left Cue', 'Right Cue', 'Sound Cue']] = df['Trial'].apply(lambda x: pd.Series(extract_trial_info(x)))

    df['Start Chamber Number'] = df['Start Chamber'].str.extract(r'Chamber (\d+)')

    #change the order of the columns 
    new_df = df[['Number', 'Start Chamber Number', 'Left Cue', 'Right Cue', 'Sound Cue', 'Result', 'Choice']]

    new_df['Trial Type'] = new_df.apply(determine_trial_type, axis=1)

    new_df.to_csv(extracted_biases_file_path, index=False)

    print(f"DataFrame saved to {extracted_biases_file_path}")

    # Group by 'Trial Types' and 'Results' and count occurrences
    grouped_df = new_df.groupby(['Trial Type', 'Result']).size().unstack(fill_value=0)

    # Rename columns for clarity
    grouped_df.columns = ['Error_Count', 'Success_Count']

    # Calculate total repetitions of each trial type
    grouped_df['Total_Repetitions'] = grouped_df.sum(axis=1)

    # Reset index to make 'Trial Types' a column again
    grouped_df = grouped_df.reset_index()

    #Create a DataFrame with all 16 trial types and zeros for counts
    all_trial_types = pd.DataFrame({
        'Trial Type': range(1, 17)
    })

    # Merge the all_trial_types DataFrame with grouped_df
    # Use a left join to ensure all trial types are included
    final_df = pd.merge(all_trial_types, grouped_df, on='Trial Type', how='left')

    # Fill NaN values resulting from the merge with 0
    final_df = final_df.fillna(0)

    # Ensure all counts are integers
    final_df['Error_Count'] = final_df['Error_Count'].astype(int)
    final_df['Success_Count'] = final_df['Success_Count'].astype(int)
    final_df['Total_Repetitions'] = final_df['Total_Repetitions'].astype(int)

    # Save the final DataFrame to CSV
    output_csv_path = os.path.join(os.path.dirname(bag_file), 'trial_type_summary.csv')
    final_df.to_csv(output_csv_path, index=False)

    print(f"Summary saved to {output_csv_path}")


past_seven_days_file_path = os.path.join(os.path.dirname(bag_file), 'Past_seven_days_biases.csv')

#check whether the analysis csv files of the past three days aleady exists. If it exists read it if not create it.
if os.path.exists(past_seven_days_file_path):
    #read the extracted bias csv file that contains trial information
    past_seven_days_biases = pd.read_csv(past_seven_days_file_path)
    print('The dataframes have been created previously')
else:
    combine_csv_files(rat_number, date)


save_plots_dir = os.path.join(os.path.dirname(bag_file), 'plots')

#check whether the plots have been created before. If not, create them.
if os.path.exists(save_plots_dir):
    print('The plots have been created previously')
else:
    trial_type_summary['Success_Rate'] = trial_type_summary['Success_Count'] / trial_type_summary['Total_Repetitions']

    # Create the 'plots' directory if it doesn't exist
    os.makedirs(save_plots_dir, exist_ok=True)

    # Plot success rate for each trial type
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Trial Type', y='Success_Rate', data=trial_type_summary)
    plt.title('Success Rate per Trial Type (Daily)')
    plt.xlabel('Trial Type')
    plt.ylabel('Success Rate')

    # Define the file name and full path
    file_name1 = 'Success Rate per Trial Type (Daily).png'
    save_path1 = os.path.join(save_plots_dir, file_name1)

    # Save the plot to the defined path
    plt.savefig(save_path1)
    plt.show()

    plt.figure(figsize=(10, 6))
    sns.barplot(x='Trial Type', y='Success_Count', data= past_seven_days_biases)
    plt.title('Success Rate per Trial Type (7-Day Cumulative)')
    plt.xlabel('Trial Type')
    plt.ylabel('Success Rate')

    # Define the file name and full path
    file_name2 = 'Success Rate per Trial Type (7-Day Cumulative).png'
    save_path2 = os.path.join(save_plots_dir, file_name2)

    # Save the plot to the defined path
    plt.savefig(save_path2)
    plt.show()

    # Plot total repetitions to see if the rat prefers certain trial types
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Trial Type', y='Total_Repetitions', data=trial_type_summary)
    plt.title('Total Repetitions per Trial Type (Daily)')
    plt.xlabel('Trial Type')
    plt.ylabel('Total Repetitions')

    # Define the file name and full path
    file_name3 = 'Total Repetitions per Trial Type (Daily).png'
    save_path3 = os.path.join(save_plots_dir, file_name3)

    # Save the plot to the defined path
    plt.savefig(save_path3)
    plt.show()

    plt.figure(figsize=(10, 6))
    sns.barplot(x='Trial Type', y='Total_Repetitions', data=past_seven_days_biases)
    plt.title('Total Repetitions per Trial Type (7-Day Cumulative)')
    plt.xlabel('Trial Type')
    plt.ylabel('Total Repetitions')

    #Define the file name and full path
    file_name4 = 'Total Repetitions per Trial Type (7-Day Cumulative).png'
    save_path4 = os.path.join(save_plots_dir, file_name4)

    # Save the plot to the defined path
    plt.savefig(save_path4)
    plt.show()

# Create a contingency table for correct vs. wrong counts across all trial types
contingency_table = past_seven_days_biases[['Success_Count', 'Error_Count']].values

# Perform chi-square test
chi2_stat, p_val, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-square Statistic: {chi2_stat}")
print(f"P-Value: {p_val}")

# Interpret the result
if p_val < 0.05:
    print("There is a statistically significant difference in performance across trial types.")
else:
    print("No statistically significant difference found across trial types.")




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
DATA_PATH is set to: \\10.34.1.59\big_gulp\nc4_rat_data\Maze_Rats
Using path: \\10.34.1.59\big_gulp\nc4_rat_data\Maze_Rats
Using path: \\10.34.1.59\big_gulp\nc4_rat_data\Maze_Rats
Looking in folder: \\10.34.1.59\big_gulp\nc4_rat_data\Maze_Rats\NC40007
Looking in folder: \\10.34.1.59\big_gulp\nc4_rat_data\Maze_Rats\NC40007\240826
The dataframes have been created previously
The dataframes have been created previously
The plots have been created previously
Chi-square Statistic: 0.9077847706018929
P-Value: 0.9999998722030669
No statistically significant difference found across trial types.
